In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import numpy as np
import matplotlib.pyplot as plt
from  tqdm import tqdm
import os
import torch.optim as optim
import idx2numpy


images_root_folder = "images-fashion-MLP-VAE"

if not os.path.exists(f"{images_root_folder}"):
    os.mkdir(f"{images_root_folder}")


# Load the labels file
trainPath = "fashion-MNIST/train-images-idx3-ubyte"
testPath = "fashion-MNIST/t10k-images-idx3-ubyte"

train = idx2numpy.convert_from_file(trainPath)
test = idx2numpy.convert_from_file(testPath)
train=torch.Tensor(train)
test=torch.Tensor(test)
train=torch.flatten(train,start_dim=1)
test=torch.flatten(test,start_dim=1)
label="fashion-MNIST/train-labels-idx1-ubyte"
labels=idx2numpy.convert_from_file(label)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
PATH="MLP-VAE-Fashion_MNIST-Dataset-Model"
EPOCHS=50
learning_rate_encoder=0.001
learning_rate_maniksoft=0.01
latent_dim=64
beta = 2

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class VariationalEnc(nn.Module):
    def __init__(self,x):
        super(VariationalEnc, self).__init__()
        self.l1=nn.Linear(784,512)
        self.l2=nn.Linear(512,x)
        self.l3=nn.Linear(512,x)
        self.dis=torch.distributions.Normal(0, 1)
        self.dis.loc = self.dis.loc.cuda() # hack to get sampling on the GPU
        self.dis.scale = self.dis.scale.cuda()
        self.kld=0
        
    
    def forward(self,x):
        x=self.l1(x)
        x = F.relu(x)
        mu=self.l2(x)
        sig=torch.exp(self.l3(x))
        z=mu + sig*self.dis.sample(mu.shape)
        self.kld=(sig**2 + mu**2 - torch.log(sig) - 1/2).sum()
        return z


class Decoder(nn.Module):
    def __init__(self,x):
        super(Decoder, self).__init__()
        self.l1=nn.Linear(x,512)
        self.l2=nn.Linear(512,784)

    def forward(self,x):
        ans=self.l1(x)
        ans=F.relu(ans)
        ans=self.l2(ans)
        ans = torch.sigmoid(ans)
        return ans



class VAE(nn.Module):
    def __init__(self,x):
        super(VAE, self).__init__()
        self.enc=VariationalEnc(x)
        self.dec=Decoder(x)

    def forward(self,x):
        x=self.enc(x)
        return self.dec(x)


def chaap2(orignal,decoded, idx, epo):
    if not os.path.exists(f"{images_root_folder}/{epo}"):
        os.mkdir(f"{images_root_folder}/{epo}")

    org1=torch.tensor(orignal)
    org1=org1.detach().cpu().numpy()
    org1=np.resize(org1,(28,28))

    dec1=torch.tensor(decoded)
    dec1=dec1.detach().cpu().numpy()
    dec1=np.resize(dec1,(28,28))

    output_path = f"{images_root_folder}/{epo}/{idx}"
    if not os.path.exists(f"{output_path}"):
        os.mkdir(f"{output_path}")

    plt.imsave(f"{output_path}/original.png" , org1, cmap='gray')
    plt.imsave(f"{output_path}/decoded.png" , dec1, cmap='gray')

In [ ]:
class Softmax(nn.Module):
    def __init__(self,x):
        super(Softmax, self).__init__()
        self.layer_64_1024 = nn.Linear(latent_dim, 1024)
        self.layer_1024_10 = nn.Linear(1024, 10)
    
    def forward(self,z):
        z = self.layer_64_1024(z)
        z = F.relu(z)
        z = self.layer_1024_10(z)
        return z

def getEncodedData(train):
    encoded_train = []
    for x in tqdm(train):
        x=x/255
        x=x.to(torch.float32)
        x = x.to(device) # GPU
        xx=manik.enc.forward(x)
        encoded_train.append(xx)
    return encoded_train
    
#manik = VAE(latent_dim).to(device)
#manik.load_state_dict(torch.load("MLP-VAE-Fashion_MNIST-Dataset-Model"))



In [ ]:
encoded_train = getEncodedData(train) 


In [ ]:
len(encoded_train)

In [ ]:
for i,x in enumerate(labels):
    encoded_train[i]=torch.cat((encoded_train[i],torch.tensor([x]).to(device)))

In [ ]:
encoded_train[0].shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [ ]:
encoded_train = [x.detach().cpu() for x in encoded_train]
encoded_train=np.array(encoded_train)


In [ ]:
encoded_train = np.array([np.array(x) for x in encoded_train])


In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(encoded_train[:,:-1], encoded_train[:,-1], test_size=0.95, random_state=42,stratify=encoded_train[:,-1])

In [ ]:
encoded_train.shape

In [ ]:
num_epochs=100
softmanik = Softmax(64).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(softmanik.parameters(), lr=0.001)
y_train = np.array(y_train,dtype=np.int32)
for epoch in tqdm(range(100)):
    for i,x in enumerate(X_train):
        x = torch.tensor(x).to(device)
        
        optimizer.zero_grad()
        outputs = softmanik.forward(x)
        loss = criterion(outputs, torch.tensor(y_train[i],dtype=torch.long).to(device))
        loss.backward()
        if (epoch + 1) % 2 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        optimizer.step()

In [ ]:
PATH="maniksoft2"
torch.save(softmanik.state_dict(), PATH)

In [ ]:

manik=VAE(latent_dim).to(device)
opt = torch.optim.Adam(manik.parameters(), lr=0.001)
# optim.SGD(net.parameters(), lr=0.001)

for i in range(EPOCHS):
    for idx,x in enumerate(tqdm(train)):
        opt.zero_grad()
        x=x/255
        x=x.to(torch.float32)
        x = x.to(device) # GPU
        xx=manik.forward(x)
#         print(xx)

        if(idx % 1000 == 0 or idx == train.shape[0]-1):
            chaap2(x,xx, idx, i)

#         print(x)

        #losses=torch.linalg.norm(xx-x,2)/784 + vaenc.enc.kld
        losses = ((x - xx)**2).sum() + beta*manik.enc.kld


        losses.backward()
        opt.step()


torch.save(manik.state_dict(), "Manik-mlp-encoder")




In [ ]:
required = torch.randn(64);
plt.figure(figsize=(1,1))

plt.imshow(vaenc.dec(required.to(device)).detach().cpu().numpy().reshape(28,28),cmap='gray')

In [ ]:
plt.figure(figsize=(1,1))
plt.imshow(train[1].reshape(28,28),cmap='gray')

In [ ]:
manik = VAE(latent_dim).to(device)
manik.load_state_dict(torch.load("MLP-VAE-Fashion_MNIST-Dataset-Model"))

In [ ]:
labels=labels.reshape(60000,-1)

In [ ]:
train=train.reshape(60000,784)

In [ ]:
train=np.hstack((train,labels))

In [ ]:
train.shape

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(train[:,:-1], train[:,-1], test_size=0.95, random_state=42,stratify=train[:,-1])

In [ ]:
X_train[0][-1]

In [ ]:
y_train.shape

In [ ]:
# plt.stairs(counts,bins)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:

model = SVC(kernel='linear', C=1.0, decision_function_shape='ovr')  # You can adjust the kernel and parameters

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data


In [ ]:
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

In [ ]:
lambda_function = lambda row:manik.enc(torch.tensor(row/255,dtype=torch.float32).to(device))
train2 =[lambda_function(row) for row in (train[:,:-1])]



In [ ]:
train2[0].shape

In [ ]:
for i in range(len(train2)):
    train2[i]=train2[i].detach().cpu().numpy()

In [ ]:
train2=np.array(train2)

In [ ]:
train2=np.hstack((train2,labels))

In [ ]:
nan_rows = np.isnan(train2).any(axis=1)
train2 = train2[~nan_rows]


In [ ]:
 X_train1, X_test1, y_train1, y_test1 = train_test_split(train2[:,:-1], train2[:,-1], test_size=0.95, random_state=42,stratify=train2[:,-1])

In [ ]:

model1 = SVC(kernel='linear', C=1.0, decision_function_shape='ovr')  # You can adjust the kernel and parameters

# Train the model
model1.fit(X_train1, y_train1)

# Make predictions on the test data


In [ ]:
y_pred1 = model1.predict(X_test1)

# Calculate accuracy
accuracy = accuracy_score(y_test1, y_pred1)
print(f"Accuracy: {accuracy}")

In [ ]:
def get_softmax_prediction(x_10_dim):
    x_10_dim = np.exp(x_10_dim - np.max(x_10_dim, keepdims=True))
    return x_10_dim / x_10_dim.sum(keepdims=True)

In [ ]:
ans=[]
for i,x in enumerate(X_test):
    input_tensor=softmanik(torch.tensor(x).to(device))
    y=torch.argmax(F.softmax(input_tensor, dim=0))
    ans.append(y.item()!=y_test[i])


In [ ]:
sum(ans)/len(ans)

In [ ]:
import numpy as np

# Example raw scores (replace this with your own data)
raw_scores = np.random.randn(10)

# Apply softmax along axis 1 to convert to probabilities across 1024 classes
def softmax(x):
    e_x = np.exp(x - np.max(x, keepdims=True))
    e_x / e_x.sum()
    mx = e_x[0]
    idx = 0
    for i in range(1,len(e_x)):
        if(e_x[i] > mx):
            idx = i
    return idx

# Apply softmax to get probabilities across 10 classes
softmax_scores = softmax(raw_scores)

# Get probabilities for the first class (index 0) to the tenth class (index 9)
probabilities_for_10_classes = softmax_scores

print("Original Raw Scores:")
print(raw_scores)

print("\nSoftmax Applied (Probabilities for 10 Classes):")
print(probabilities_for_10_classes)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Create a simple neural network class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(64, 64*1024)
        self.fc2 = nn.Linear(64*1024, 1024*10)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

# Training loop
num_epochs = 1000

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = net(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Training finished!')

# Now you can use the trained network for predictions.


In [ ]:
manik = VAE(latent_dim).to(device)
manik.load_state_dict(torch.load("MLP-VAE-Fashion_MNIST-Dataset-Model"))